In [132]:
from numpy.random import normal
from numpy import float16
from math import floor, log2

In [156]:
# Random array generator
def random_array(length, normalize=False):
    array = normal(loc=0.0, scale=6, size=(1,length)).astype(float16)[0]

    if normalize:
        max_num = max(array)
        i = 0
        while i < len(array):
            array[i] = array[i]/max_num
            i += 1
    else:
        i = 0
        while i<len(array):
            if array[i] > 0 and array[i] < 1:
                array[i] += 1
            elif array[i] < 0 and array[i] >-1:
                array[i] -= 1
            i += 1
    return array

def split_array_into_chunks(arr, chunk_size):
    num_chunks = (len(arr) + chunk_size - 1) // chunk_size # calculate the number of chunks
    chunks = [[] for _ in range(num_chunks)] # create empty list of chunks

    for i, elem in enumerate(arr):
        chunk_index = i // chunk_size # calculate the index of the current chunk
        chunks[chunk_index].append(elem) # add the current element to the current chunk

    return chunks

def get_shared_bias(BM_chunk, e):
    Es = [floor(log2(abs(elem))) for elem in BM_chunk]
    return max(Es) - (2**(e-1)-1)

def get_shared_bias_testing(BM_chunk, e):
    Es_abs = [abs(elem) for elem in BM_chunk]
    Es_log2 = [log2(elem) for elem in Es_abs]
    Es = [floor(elem) for elem in Es_log2]
    shared_bais = max(Es) - (2**(e-1)-1)
    print(f"Es abs are {Es_abs}")
    print(f"Es log2 are {Es_log2}")
    print(f"Exponents are {Es}, 2**(e-1)-1 is {2**(e-1)-1}")
    print(f"shared_bias is {shared_bais}")
    return shared_bais

def to_twos_complement(n:int, width):
    # Determine the minimum number of bits required to represent n in 2's complement form
    # width = n.bit_length() + 1
    
    # Convert to binary with prefix '0b'
    bin_str = bin(n & int("1"*width, 2))[2:]
    
    # If the number is negative, subtract 2**width to obtain its 2's complement
    if bin_str[0] == 'b':
        return bin(n & int("1"*width, 2) - (1 << len(bin_str)))[2:]
    else:
        return '0'*(width - len(bin_str)) + bin_str

def decimal_to_binary(fraction):
    if fraction == 0:
        return "0.0"
 
    sign = '-' if fraction < 0 else ''
    fraction = abs(fraction)
    integer_part = int(fraction)
    fractional_part = fraction - integer_part
    integer_binary = bin(integer_part)[2:]
    fractional_binary = ''
 
    while fractional_part != 0:
        fractional_part *= 2
        if fractional_part >= 1:
            fractional_binary += '1'
            fractional_part -= 1
        else:
            fractional_binary += '0'
    
    result = sign + integer_binary + '.' + fractional_binary
    return result

def fractional_to_BM(frac: str, shared_bias, e, m):
    # sign
    if frac[0] == '-':
        sign = '1'
        frac = frac[1:]
    else:
        sign = '0'

    # exponent
    integer_binary, fractional_binary = frac.split('.')
    actual_exponent = len(integer_binary)-1
    exponent_int = actual_exponent + shared_bias
    exponent = to_twos_complement(exponent_int, e)
    if exponent_int.bit_length() + 1 > e:
        raise OverflowError(f"Not enough exponent bits, exponent overflowed! fraction number is {frac}, shared bias is {shared_bias}, actual exponent is {actual_exponent} and exponent is {exponent_int}, {exponent}")

    # mantissa
    mantissa = integer_binary + fractional_binary
    mantissa = mantissa[1:]
    if len(mantissa) <= m:
        mantissa = mantissa + (m-len(mantissa))*'0'
    else:
        mantissa = mantissa[0:m]

    BM = sign + exponent + mantissa
    return BM

def dot_product(array_1, array_2):
    if len(array_1) != len(array_2):
        raise ValueError("Arrays must have equal length")
    return sum(x * y for x, y in zip(array_1, array_2))


def binary_fraction_to_decimal(binary_fraction):
    # Check if the number is negative
    is_negative = False
    if binary_fraction[0] == '-':
        is_negative = True
        binary_fraction = binary_fraction[1:]
    
    # Separate the integer and fractional parts
    integer_part, fractional_part = binary_fraction.split('.')
    
    # Convert the integer part to decimal
    decimal = 0
    power = len(integer_part) - 1
    for bit in integer_part:
        if bit == '1':
            decimal += 2 ** power
        power -= 1
    
    # Convert the fractional part to decimal
    power = -1
    for bit in fractional_part:
        if bit == '1':
            decimal += 2 ** power
        power -= 1
    
    # Apply negative sign if necessary
    if is_negative:
        decimal = -decimal
    
    return decimal


# Encapsulate BM generator into a function

In [134]:
def BM_generator():
    BM_SIZE = 8
    NUM_DATA_IN_BLOCK = 8
    ARRAY_LEN = 32
    e = 3
    m = BM_SIZE - 1 - e
    normalize = False

    fp_array = random_array(ARRAY_LEN, normalize)
    chunks = split_array_into_chunks(fp_array, NUM_DATA_IN_BLOCK)
    BMs = []
    for chunk in chunks:
        shared_bias = get_shared_bias(chunk, e)
        Binary_chunk = [decimal_to_binary(frac) for frac in chunk]
        BM_chunk = [fractional_to_BM(Bin, shared_bias, e, m) for Bin in Binary_chunk]
        BMs.append((BM_chunk, shared_bias))
    return fp_array, chunks, BMs

def BM_printer(BMs, chunks):
    for i, BM in enumerate(BMs):
        print(f"BM number {i+1}:")
        print(f"Shared bias is {BM[1]}")
        print("The block minifloats are:")
        print("\n".join(BM[0]))
        print("Their corresponding decimal forms are:")
        print(chunks[i])
        print(*chunks[i], sep='\n')
        print()

def verilog_code_generator(BMs_1, BMs_2):
    file = "G:\\OneDrive - The University of Sydney (Students)\\semester1 2023\\capstone\\python code\\verilog_testing_code.txt"
    with open(file, 'w') as f:
        i = 1
        for BM1_tuple, BM2_tuple in zip(BMs_1, BMs_2):
            f.write(f"Testing data {i}\n")
            i += 1
            f.write(f"sb1 = {BM1_tuple[1]};\n")
            f.write(f"sb2 = {BM2_tuple[1]};\n\n")
            f.write("#5 reset = 1'b1;\n")
            f.write("#5 reset = 1'b0;    #15\n\n")
            loop_idx = 1
            for BM1, BM2 in  zip(BM1_tuple[0], BM2_tuple[0]):
                bm1_len = str(len(BM1))
                bm2_len = str(len(BM2))
                if loop_idx == 1:
                    f.write(f"in1 = {bm1_len}'b" + BM1 + ";\n")
                    f.write(f"in2 = {bm2_len}'b" + BM2 + ";\n")
                    f.write("#10 write_enable = 1'b1;\n\n")
                    loop_idx += 1
                elif loop_idx == 2:
                    f.write("#15\n")
                    f.write(f"in1 = {bm1_len}'b" + BM1 + ";\n")
                    f.write(f"in2 = {bm2_len}'b" + BM2 + ";\n\n")
                    loop_idx = 0
                else:
                    f.write("#25\n")
                    f.write(f"in1 = {bm1_len}'b" + BM1 + ";\n")
                    f.write(f"in2 = {bm2_len}'b" + BM2 + ";\n\n")
            f.write("\n\n\n")

# BM generator test

In [4]:
# Constents
BM_SIZE = 8
NUM_DATA_IN_BLOCK = 8
e = 3
m = BM_SIZE - 1 - e

In [5]:
# Random array
fp_array1 = random_array(32, False)
print(fp_array1)

[ 4.492   1.353   8.13   -4.      0.261  -4.094   1.116   1.718  11.54
 -3.584   0.9907  7.758  -2.824   2.65   -6.38   -2.803  -6.797  -4.496
 -2.832   5.074  -3.547   3.283  -1.88    0.6934 -1.941   5.254   6.246
 -1.621  -3.39    7.496   3.32    0.6104]


In [23]:
chunks = split_array_into_chunks(fp_array1, NUM_DATA_IN_BLOCK)
print('\n'.join(map(str, chunks)))

[4.492, 1.353, 8.13, -4.0, 0.261, -4.094, 1.116, 1.718]
[11.54, -3.584, 0.9907, 7.758, -2.824, 2.65, -6.38, -2.803]
[-6.797, -4.496, -2.832, 5.074, -3.547, 3.283, -1.88, 0.6934]
[-1.941, 5.254, 6.246, -1.621, -3.39, 7.496, 3.32, 0.6104]


In [39]:
# convert all the chunks into BM
BMs = []
for chunk in chunks:
    shared_bias = get_shared_bias(chunk, e)
    Binary_chunk = [decimal_to_binary(frac) for frac in chunk]
    BM_chunk = [fractional_to_BM(Bin, shared_bias, e, m) for Bin in Binary_chunk]
    BMs.append((BM_chunk, shared_bias))

In [ ]:
# print the result in proper form to feed into verilog design
for i, BM in enumerate(BMs):
    print(f"BM number {i+1}:")
    print(f"Shared bias is {BM[1]}")
    print("The block minifloats are:")
    print("\n".join(BM[0]))
    print("Their corresponding decimal forms are:")
    print(chunks[i])
    print()


# Run the BM generator

In [171]:
fp1, fp1_chunked, BM1 = BM_generator()
fp2, fp2_chunked, BM2 = BM_generator()

dps = []
for chunk1, chunk2 in zip(fp1_chunked, fp2_chunked):
    dps.append(dot_product(chunk1, chunk2))
print("Chunks dot product should be")
print(*dps, sep='\n')

verilog_code_generator(BM1, BM2)

Chunks dot product should be
88.9560546875
-135.703125
-5.13671875
-203.890625


In [172]:
BM_printer(BM1, fp1_chunked)
BM_printer(BM2, fp2_chunked)
# dp = dot_product(fp1, fp2)
# print(f"The dot product of these two arrays should be {dp}")

BM number 1:
Shared bias is 0
The block minifloats are:
00010111
10100010
10001001
10001100
10000010
00101100
10110011
10110011
Their corresponding decimal forms are:
[2.953, -4.555, -1.572, -1.791, -1.162, 7.215, -9.984, -9.68]
2.953
-4.555
-1.572
-1.791
-1.162
7.215
-9.984
-9.68

BM number 2:
Shared bias is -1
The block minifloats are:
10010111
10001111
01110010
00000110
01111100
00010110
11110010
00010101
Their corresponding decimal forms are:
[-5.895, -3.955, 1.174, 2.85, 1.793, 5.527, -1.179, 5.395]
-5.895
-3.955
1.174
2.85
1.793
5.527
-1.179
5.395

BM number 3:
Shared bias is -1
The block minifloats are:
11111010
00010111
00010011
00010110
01110100
00011000
10001101
00010001
Their corresponding decimal forms are:
[-1.625, 5.977, 4.836, 5.527, 1.254, 6.227, -3.729, 4.324]
-1.625
5.977
4.836
5.527
1.254
6.227
-3.729
4.324

BM number 4:
Shared bias is 0
The block minifloats are:
00000000
10001110
10110101
00110110
10001110
00010100
00001110
00100000
Their corresponding decimal forms

### Result checking

In [10]:
tem_dp = dot_product(fp1_chunked[0], fp1_chunked[1])
print(tem_dp)

8.427734375


In [11]:
binary_fraction = '1010111000.0'
decimal = binary_fraction_to_decimal(binary_fraction)
print(decimal)

696


In [32]:
print('\n'.join(map(str, fp1_chunked)))

[-3.834, 1.111, -4.57, 4.824, -6.26, -8.195, 0.1362, -0.7227]
[-6.83, -1.873, -0.01903, 2.486, 5.527, -5.02, -1.357, -1.397]
[6.703, 3.545, -0.04977, 0.3894, 4.914, -2.62, -0.6836, 6.0]
[-9.98, 0.0285, 6.86, -5.375, 3.29, -3.988, 9.3, 8.41]


In [33]:
print(BM1)
print(fp1_chunked)

[(['10011110', '00000001', '10100010', '00100011', '10101001', '10110000', '00000010', '10001011'], 0), (['10011011', '10111101', '10110000', '00000011', '00010110', '10010100', '10110101', '10110110'], -1), (['00011010', '00001100', '10110000', '00110110', '00010011', '10000100', '10111010', '00011000'], -1), (['10110011', '00000000', '00101011', '10100101', '00011010', '10011111', '00110010', '00110000'], 0)]
[[-3.834, 1.111, -4.57, 4.824, -6.26, -8.195, 0.1362, -0.7227], [-6.83, -1.873, -0.01903, 2.486, 5.527, -5.02, -1.357, -1.397], [6.703, 3.545, -0.04977, 0.3894, 4.914, -2.62, -0.6836, 6.0], [-9.98, 0.0285, 6.86, -5.375, 3.29, -3.988, 9.3, 8.41]]


# Unit tests

In [ ]:
def test_bm_printer():
    bm = [(['10011110', '00000001', '10100010', '00100011', '10101001', '10110000', '00000010', '10001011'], 0), (['10011011', '10111101', '10110000', '00000011', '00010110', '10010100', '10110101', '10110110'], -1), (['00011010', '00001100', '10110000', '00110110', '00010011', '10000100', '10111010', '00011000'], -1), (['10110011', '00000000', '00101011', '10100101', '00011010', '10011111', '00110010', '00110000'], 0)]
    fp = [[-3.834, 1.111, -4.57, 4.824, -6.26, -8.195, 0.1362, -0.7227], [-6.83, -1.873, -0.01903, 2.486, 5.527, -5.02, -1.357, -1.397], [6.703, 3.545, -0.04977, 0.3894, 4.914, -2.62, -0.6836, 6.0], [-9.98, 0.0285, 6.86, -5.375, 3.29, -3.988, 9.3, 8.41]]
    BM_printer(bm, fp)
test_bm_printer()

In [27]:
def test_twos_complement():
    U_1 = to_twos_complement(-1, 3)
    U_2 = to_twos_complement(-15, 5)
    print(U_1)
    print(U_2)
test_twos_complement()

111
10001


In [14]:
def test_get_shared_bias():
    U_e = 3
    U_block = [2.725405629730855, 7.756529614174363, -0.06001146100609999, 0.2990818474643237, -0.9200318592211703, 0.729987441367691, 6.072331412966728, 1.2784195577767334]
    get_shared_bias_testing(chunks[0], U_e)

In [15]:
def test_fractional_to_bm():
    U_BM = fractional_to_BM('101.11011', -3, 3, 4)
    print(U_BM)

In [16]:
def test_decimal_to_binary():
    fraction = 1.25
    binary = decimal_to_binary(fraction)
    print("Binary representation of", fraction, "is", binary)

In [17]:
def test_dot_product():
    a1 = [1,2,3]
    a2 = [1,1,1]
    DotProduct = [a*b for a,b in zip(a1, a2)]
    Sum = sum(DotProduct)
    print(DotProduct)
    print(Sum)

In [18]:
def test_decimal_to_binary():
    sum_binary = decimal_to_binary(Sum)
    print(sum_binary)

In [19]:
def test_algorithm_frac_to_bm():
    e = 3
    m = 4
    fp_numbers = [3.123, -6.55, -0.7764, -5.766, -6.938, 9.87, -1.545, -13.375]
    shared_bias = get_shared_bias(fp_numbers, e)
    bin_number = decimal_to_binary(7.28)
    bm_number = fractional_to_BM(bin_number, -1, e, m)

    print(shared_bias, bin_number, bm_number, sep='\n')
# test_algorithm_frac_to_bm()
    

# Tools

In [71]:
def Tool_bin2dec():
    binary_fraction = '11.011'
    print(binary_fraction_to_decimal(binary_fraction))
Tool_bin2dec()

3.375


In [14]:
def Tool_dec2bin():
    decs = [0.1, 0.25, 0.5, 0.75, 1, 1.25, 1.5]
    bin = [decimal_to_binary(dec) for dec in decs]
    print(bin)
Tool_dec2bin()

['0.0001100110011001100110011001100110011001100110011001101', '0.01', '0.1', '0.11', '1.', '1.01', '1.1']


In [106]:
def Tool_dot_product():
    dec_array_1 = [-3.86, 0.511, 2.234, 0.1027, 3.3, -2.242, -1.0625, -2.75]
    dec_array_2 = [-11.2, -4.49, 2.236, 1.288, 0.498, -3.307, 3.63, -4.395]
    dot_product = sum(x*y for x,y in zip(dec_array_1, dec_array_2))
    print(dot_product)
Tool_dot_product()
    

63.3521806


In [155]:
import sys, os
def Tool_BM_reader(file_in, file_out):
    e = 3
    m = 4
    E = 8
    M = 23
    power1 = -(2*m+2)
    shared_bias = 0

    path = sys.path[0]
    # path = 'G:\\OneDrive - The University of Sydney (Students)\\semester1 2023\\capstone\\python code\\'
    input_file = os.path.join(path, file_in)
    output_file = os.path.join(path, file_out)
    
    with open(input_file, 'r') as fi:
        dec = []

        for line in fi:
            if line == '\n':
                dec.append('')
                continue
            BM = line.rstrip('\n')
            
            sign = '' if BM[0] == '0' else '-' 
            exp = BM[1:E+1]
            man = sign + BM[E+1:]

            exp_int = int(exp, 2) - shared_bias
            power = power1 + exp_int

            man_shifted = man[:power] + '.' + man[power:]

            dec.append(binary_fraction_to_decimal(man_shifted))
    with open(output_file, 'w') as fo:
        for num in dec:
            fo.write(str(num) + '\n')

Tool_BM_reader("BMs.txt", "decs.txt")